In [9]:
import random

# --- Datos del problema ---
tasks = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
precedence = {
    'C': ['A'],
    'D': ['A'],
    'E': ['B'],
    'F': ['C', 'D'],
    'G': ['E', 'F'],
    'H': ['G'],
    'I': ['F'],
    'J': ['H', 'I']
}

def is_valid(order):
    position = {task: i for i, task in enumerate(order)}
    for task, preds in precedence.items():
        for pred in preds:
            if position[pred] > position[task]:
                return False
    return True

def fitness(order):
    position = {task: i for i, task in enumerate(order)}
    score = 100
    for task, preds in precedence.items():
        for pred in preds:
            if position[pred] > position[task]:
                score -= 30
            else:
                # Penalizar distancia grande entre pred y task
                score -= (position[task] - position[pred] - 1) * 2
    return score


## Generar población

In [10]:
def create_individual():
    return random.sample(tasks, len(tasks))

def create_population(size=20):
    return [create_individual() for _ in range(size)]

population = create_population()
for ind in population:
    print(ind, "-> Fitness:", fitness(ind))


['A', 'E', 'G', 'I', 'J', 'D', 'H', 'C', 'B', 'F'] -> Fitness: -54
['J', 'C', 'E', 'I', 'A', 'D', 'B', 'F', 'G', 'H'] -> Fitness: -72
['F', 'G', 'D', 'I', 'A', 'E', 'J', 'H', 'B', 'C'] -> Fitness: -106
['E', 'F', 'C', 'G', 'B', 'D', 'A', 'J', 'I', 'H'] -> Fitness: -138
['E', 'F', 'D', 'H', 'G', 'B', 'A', 'J', 'C', 'I'] -> Fitness: -112
['C', 'A', 'J', 'D', 'G', 'I', 'H', 'F', 'B', 'E'] -> Fitness: -102
['H', 'C', 'G', 'J', 'A', 'I', 'B', 'E', 'D', 'F'] -> Fitness: -104
['E', 'F', 'I', 'G', 'H', 'A', 'B', 'D', 'C', 'J'] -> Fitness: -22
['J', 'E', 'I', 'H', 'C', 'D', 'G', 'F', 'A', 'B'] -> Fitness: -154
['D', 'B', 'H', 'I', 'E', 'C', 'J', 'A', 'F', 'G'] -> Fitness: -60
['D', 'H', 'B', 'C', 'A', 'F', 'J', 'E', 'G', 'I'] -> Fitness: -56
['I', 'E', 'F', 'C', 'A', 'J', 'G', 'B', 'D', 'H'] -> Fitness: -112
['J', 'A', 'H', 'E', 'B', 'D', 'I', 'C', 'F', 'G'] -> Fitness: -80
['B', 'A', 'E', 'D', 'G', 'F', 'C', 'J', 'I', 'H'] -> Fitness: -48
['F', 'A', 'D', 'E', 'B', 'J', 'H', 'G', 'C', 'I'] -> F

## Operadores GA: selección, cruce, mutación

In [11]:
def selection(pop):
    a, b = random.sample(pop, 2)
    return a if fitness(a) > fitness(b) else b

def crossover(p1, p2):
    size = len(p1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None]*size
    child[start:end] = p1[start:end]
    fill = [x for x in p2 if x not in child]
    idx = 0
    for i in range(size):
        if child[i] is None:
            child[i] = fill[idx]
            idx += 1
    return child

def mutate(ind, prob=0.2):
    if random.random() < prob:
        i, j = random.sample(range(len(ind)), 2)
        ind[i], ind[j] = ind[j], ind[i]
    return ind


## Evolución

In [12]:
def evolve(population, generations=50):
    for gen in range(generations):
        new_pop = []
        for _ in range(len(population)):
            p1 = selection(population)
            p2 = selection(population)
            child = crossover(p1, p2)
            child = mutate(child)
            new_pop.append(child)
        population = new_pop
        best = max(population, key=fitness)
        print(f"Gen {gen+1:2d}: {best} | Fitness: {fitness(best)} | Válido: {is_valid(best)}")
    return best

best = evolve(population)
print("\nMejor solución encontrada:", best)


Gen  1: ['A', 'B', 'H', 'I', 'E', 'C', 'J', 'D', 'F', 'G'] | Fitness: -6 | Válido: False
Gen  2: ['A', 'B', 'D', 'E', 'H', 'I', 'C', 'J', 'F', 'G'] | Fitness: -2 | Válido: False
Gen  3: ['A', 'D', 'H', 'I', 'C', 'B', 'E', 'J', 'F', 'G'] | Fitness: -2 | Válido: False
Gen  4: ['F', 'A', 'I', 'B', 'E', 'G', 'D', 'H', 'C', 'J'] | Fitness: -6 | Válido: False
Gen  5: ['B', 'I', 'A', 'E', 'C', 'D', 'F', 'J', 'G', 'H'] | Fitness: 8 | Válido: False
Gen  6: ['B', 'I', 'A', 'E', 'C', 'D', 'F', 'J', 'G', 'H'] | Fitness: 8 | Válido: False
Gen  7: ['A', 'I', 'B', 'E', 'C', 'D', 'F', 'G', 'H', 'J'] | Fitness: 34 | Válido: False
Gen  8: ['C', 'D', 'F', 'B', 'I', 'A', 'E', 'G', 'H', 'J'] | Fitness: 16 | Válido: False
Gen  9: ['C', 'D', 'F', 'B', 'I', 'A', 'E', 'G', 'H', 'J'] | Fitness: 16 | Válido: False
Gen 10: ['I', 'A', 'B', 'C', 'E', 'D', 'F', 'G', 'H', 'J'] | Fitness: 36 | Válido: False
Gen 11: ['A', 'B', 'C', 'E', 'D', 'F', 'I', 'G', 'H', 'J'] | Fitness: 74 | Válido: True
Gen 12: ['A', 'B', 'C', 